In [1]:
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

sc = SparkContext()
sqlContext = SQLContext(sc)

movie = pd.read_csv(r'/home/kritz/Documents/DDL/Ex09/movies.csv')
rating = pd.read_csv(r'/home/kritz/Documents/DDL/Ex09/ratings.csv')

In [6]:
movie['title']= movie['title'].astype(str)
movie['genres']= movie['genres'].astype(str)

movie = sqlContext.createDataFrame(movie)
rating = sqlContext.createDataFrame(rating)

In [10]:
data = movie.join(rating, "movieId" ,"inner")
data.show(5)

+-------+--------------+------+------+------+----------+
|movieId|         title|genres|userId|rating| timestamp|
+-------+--------------+------+------+------+----------+
|     26|Othello (1995)| Drama|     9|   3.0| 938628655|
|     26|Othello (1995)| Drama|    70|   5.0| 853954729|
|     26|Othello (1995)| Drama|    86|   4.0| 848160245|
|     26|Othello (1995)| Drama|   119|   5.0| 913117867|
|     26|Othello (1995)| Drama|   294|   3.5|1082754003|
+-------+--------------+------+------+------+----------+
only showing top 5 rows



In [14]:
columns_to_drop = ['movieId', 'genres','userId','timestamp']
test = data.drop(*columns_to_drop)
test.show(10)

+--------------------+------+
|               title|rating|
+--------------------+------+
|      Othello (1995)|   3.0|
|      Othello (1995)|   5.0|
|      Othello (1995)|   4.0|
|      Othello (1995)|   5.0|
|      Othello (1995)|   3.5|
|City of Lost Chil...|   4.5|
|City of Lost Chil...|   3.0|
|City of Lost Chil...|   4.0|
|City of Lost Chil...|   4.0|
|City of Lost Chil...|   4.0|
+--------------------+------+
only showing top 10 rows



In [39]:
final = test.rdd.groupByKey().mapValues(lambda x: sum(x) / len(x)).max(key=lambda x:x[1])
print(final)

('Even Dwarfs Started Small (Auch Zwerge haben klein angefangen) (1971)', 5.0)


In [44]:
from pyspark.sql.functions import countDistinct
df = data.groupby('userId').agg(countDistinct("movieId"))
filtered = df.filter(df['count(DISTINCT movieId)'] > 40)

In [48]:
newData = data.join(filtered, "userId" ,"inner")
columns_to_drop = ['movieId','title','genres','count(DISTINCT movieId)','timestamp']
newData = newData.drop(*columns_to_drop)
newData.show()

+------+------+
|userId|rating|
+------+------+
|    26|   4.0|
|    26|   2.0|
|    26|   2.0|
|    26|   3.0|
|    26|   3.5|
|    26|   3.5|
|    26|   4.0|
|    26|   2.0|
|    26|   3.0|
|    26|   4.0|
|    26|   4.0|
|    26|   3.5|
|    26|   5.0|
|    26|   4.5|
|    26|   3.0|
|    26|   3.0|
|    26|   3.5|
|    26|   4.5|
|    26|   3.5|
|    26|   3.5|
+------+------+
only showing top 20 rows



In [49]:
user = newData.rdd.groupByKey().mapValues(lambda x: sum(x) / len(x)).min(key=lambda x:x[1])
print(user)

(581, 1.4591836734693877)


In [50]:
columns_to_drop = ['movieId', 'title','userId','timestamp']
genre = data.drop(*columns_to_drop)
genre.show()

+--------------------+------+
|              genres|rating|
+--------------------+------+
|               Drama|   3.0|
|               Drama|   5.0|
|               Drama|   4.0|
|               Drama|   5.0|
|               Drama|   3.5|
|Adventure|Drama|F...|   4.5|
|Adventure|Drama|F...|   3.0|
|Adventure|Drama|F...|   4.0|
|Adventure|Drama|F...|   4.0|
|Adventure|Drama|F...|   4.0|
|Adventure|Drama|F...|   5.0|
|Adventure|Drama|F...|   5.0|
|Adventure|Drama|F...|   3.0|
|Adventure|Drama|F...|   5.0|
|Adventure|Drama|F...|   3.5|
|Adventure|Drama|F...|   2.5|
|Adventure|Drama|F...|   5.0|
|Adventure|Drama|F...|   5.0|
|Adventure|Drama|F...|   4.0|
|Adventure|Drama|F...|   5.0|
+--------------------+------+
only showing top 20 rows



In [51]:
movieGenre = genre.rdd.groupByKey().mapValues(lambda x: sum(x) / len(x)).max(key=lambda x:x[1])
print(movieGenre)

('Action|Comedy|Drama|Romance', 5.0)
